<font size="10" color="black">Amostragem</font>

Eduardo Chaves Ferreira

## Importação de bibliotecas usadas nos exemplos

In [ ]:


import numpy as np
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
import scipy.stats as stat
from collections import Counter
from scipy.stats import t
from scipy.stats import norm

import os

path = os.environ['PATH']

if path.startswith('C'):
    IN_KAGGLE = False
else:
    IN_KAGGLE = True

## Mantendo a reprodutibilidade dos resultados

Antes da geração de números aleatórios é importante inicializar o gerador de números para que os resultados sejam os mesmos

In [ ]:
# Para uso com funções da biblioteca standard (ex random.randint)
random.seed(1)
# Para uso com funções da biblioteca numpy (ex np.random.randint)
np.random.seed(1)

# Quando for passada como parâmetro a seed
random_state = 1

## Funções gerais usadas nos exemplos

In [ ]:
# Calcula número de desvios na distribuição t de student para determinada confianca


def calcula_Z_tstudent(confianca, tamanho_amostra):
    mean = 0
    std = 1
    rv = t(df=(tamanho_amostra-1))
    return rv.interval(confianca)[1]

In [ ]:
# Calcula número de desvios na distribuição normal para determinada confianca


def calcula_Z_normal(confianca):
    mean = 0
    std = 1
    rv = norm(loc=mean, scale=std)
    return rv.interval(confianca)[1]

In [ ]:
# Calcula número de desvios na distribuição normal para determinada confianca e múltiplas proporções

def calcula_Z_Multiplas_Proporcoes(confianca, numero_proporcoes):
    area = math.pow(  confianca, 1/(numero_proporcoes-1) )
    return calcula_Z_normal(area)



In [ ]:
# Recupera amostra da população

def recupera_amostra(populacao, tamanho_amostra):
    tamanho_amostra = int(tamanho_amostra)
    if type(populacao) is pd.DataFrame:
        return populacao.sample(tamanho_amostra)
    else:
        #return populacao[np.random.randint(0, len(populacao), tamanho_amostra)]
        return [populacao[i] for i in np.random.randint(0, len(populacao), tamanho_amostra)]

In [ ]:
# Função que calcula intervalo de confiança para média com base em uma amostra e a confiança desejada

def calcula_intervalo_media(amostra, confianca, tamanho_populacao):
    tamanho_amostra = len(amostra)
    
    #1-Calcule a média da amostra ex. media_amostra = np.mean(amostra)
    media_amostra = np.mean(amostra)

    #2-Calcule o desvio da amostra ex. desvio_amostra = np.std(amostra)
    desvio_amostra = np.std(amostra)

    #3-Calcule quantos desvios precisará para seu grau de confiânça ex. numero_desvios = calcula_numero_desvios_tstudent_para_confianca(confiança, tamanho_amostra)
    numero_desvios = calcula_Z_tstudent(confianca, tamanho_amostra)

    #4-Calcule o desvio das amostras ex. desvio_amostras = desvio_amostra/np.sqrt(tamanho_amostra)
    desvio_amostras = desvio_amostra/np.sqrt(tamanho_amostra)

    #5-Calcule a margem de erro ex. margem_erro = numero_desvios*desvio_amostras
    margem_erro = numero_desvios*desvio_amostras
    
    if tamanho_amostra>0.05*tamanho_populacao:
        margem_erro = margem_erro * (np.sqrt(tamanho_populacao-tamanho_amostra)/np.sqrt(tamanho_populacao-1))

    #6-Calcule o intervalo ex. inferior = media_amostra-margem_erro, superior = media_amostra+margem_erro
    inferior = media_amostra-margem_erro
    superior = media_amostra+margem_erro

    return inferior,superior

In [ ]:
# Função que calcula intervalo de confiança para proporção de determinado valor com base em uma amostra e a confiança desejada
# O parâmetro valor representa a categoria na amostra para a qual se deseja o intervalo de confiança da proporção, por exemplo o nome de um candidato

def calcula_intervalo_proporcao(amostra, confianca, valor, tamanho_populacao):
    tamanho_amostra = len(amostra)
    
    proporcao_valor = Counter(amostra)[valor]/tamanho_amostra
    
    numero_desvios = calcula_Z_tstudent(confianca, tamanho_amostra)

    margem_erro = numero_desvios*np.sqrt(proporcao_valor*(1-proporcao_valor))/np.sqrt(tamanho_amostra)
    
    if tamanho_amostra>0.05*tamanho_populacao:
        margem_erro = margem_erro * (np.sqrt(tamanho_populacao-tamanho_amostra)/np.sqrt(tamanho_populacao-1))

    inferior = proporcao_valor-margem_erro
    superior = proporcao_valor+margem_erro

    return inferior,superior

In [ ]:
# Função para cálculo do número de amostras para médias e somas

def Tamanho_Amostra_Valor_Pontual(Tamanho_Populacao, Numero_Desvios, Variancia, Margem_Erro):
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Numero_Desvios,2) * Variancia) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Numero_Desvios,2)*Variancia )
            )

In [ ]:
# Função para cálculo do número de amostras para proporções simples

def Tamanho_Amostra_Proporcao_Simples(Tamanho_Populacao, Numero_Desvios, Proporcao, Margem_Erro):
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Numero_Desvios,2) * Proporcao * (1-Proporcao)) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Numero_Desvios,2)*Proporcao*(1-Proporcao) )
            )

In [ ]:
# Função para cálculo do tamanho da amostra para proporções múltiplas

def Tamanho_Amostra_Proporcao_Multipla(Tamanho_Populacao, Confianca, Numero_Proporcoes, Margem_Erro):
    Confianca_ = calcula_Z_Multiplas_Proporcoes(Confianca, Numero_Proporcoes)
    return math.ceil( 
            (Tamanho_Populacao * math.pow(Confianca_,2) * 0.25 ) / \
            ( (Tamanho_Populacao-1)*math.pow(Margem_Erro,2) + math.pow(Confianca_,2)*0.25 )
            )

<font size="6" color="red">Geração de dados simulando toda a população</font>

In [ ]:
Tamanho_Populacao = 2057
Tamanho_Populacao_Clinicas_Publicas = 909
Tamanho_Populacao_Clinicas_Privadas = 1148

In [ ]:
# Cria estrutura vazia para guadar dados da população
populacao = pd.DataFrame( columns=['Area','Procurou_Outra_Clinica','Dias_Ate_Atendimento','Satisfacao'])

# Divide entre clínicas públicas e privadas

In [ ]:
#Distribui pacientes em clínicas públicas e privadas
populacao.Area = ['Publico']*909 + ['Privado']*1148
populacao.head()

In [ ]:
populacao['Area'].value_counts().plot(kind='bar')
plt.xlabel('Area')
plt.ylabel('Quantidade')

plt.title('Area')
plt.grid(True)
plt.show()

# Procurou outra clínica?

In [ ]:
# 218 pacientes da pública procuraram outra clínica (24%), 300 da privada procuraram (26,1%)
populacao.Procurou_Outra_Clinica = ['S']*218 + ['N']*691 + ['S']*300 + ['N']*848
populacao.head()

In [ ]:
populacao['Procurou_Outra_Clinica'].value_counts().plot(kind='bar')
plt.xlabel('InteProcurou_Outra_Clinicaresse')
plt.ylabel('Quantidade')

plt.title('Procurou_Outra_Clinica')
plt.grid(True)
plt.show()

print(sum(populacao.Procurou_Outra_Clinica == 'S')/Tamanho_Populacao)

# Dias até atendimento

In [ ]:
# Dias para atendimento média 71 pública, 53 na privada, desvio 22.5 ambas
rv_pub = norm(loc=71, scale=18.73)
rv_pri = norm(loc=53, scale=14.69)
populacao.Dias_Ate_Atendimento =  list(rv_pub.rvs(size=909)) + list(rv_pri.rvs(size=1148))
populacao.Dias_Ate_Atendimento = populacao.Dias_Ate_Atendimento.astype('int')
populacao.loc[populacao.Dias_Ate_Atendimento <1,['Dias_Ate_Atendimento']] = 1
populacao.head()

In [ ]:
n, bins, patches = plt.hist(populacao.Dias_Ate_Atendimento,  facecolor='g', alpha=0.75, bins=50)
plt.xlabel('Dias_Ate_Atendimento')
plt.ylabel('Quantidade')

plt.title('Dias_Ate_Atendimento')
plt.grid(True)
plt.show()

print(populacao.Dias_Ate_Atendimento.mean())

# Satisfação com atendimento

In [ ]:
# Grau de satisfação arbitrado
populacao.Satisfacao = random.choices(population=['I','R','S'], weights=[0.2,0.5,0.3], k=2057)
populacao.head()

In [ ]:
populacao['Satisfacao'].value_counts().plot(kind='bar')
plt.xlabel('Satisfacao')
plt.ylabel('Quantidade')

plt.title('Satisfacao')
plt.grid(True)
plt.show()

print(sum(populacao.Satisfacao == 'I')/Tamanho_Populacao)

# Embaralha registros

In [ ]:
from sklearn.utils import shuffle
populacao = shuffle(populacao)
populacao.head()

# Separa a população em duas sub populações: atendidos por clínicas públicas e privadas

In [ ]:
Populacao_Clinicas_Publicas = populacao.loc[populacao.Area == 'Publico',:]
Tamanho_Populacao_Clinicas_Publicas = len(Populacao_Clinicas_Publicas)
Populacao_Clinicas_Privadas = populacao.loc[populacao.Area == 'Privado',:]
Tamanho_Populacao_Clinicas_Privadas = len(Populacao_Clinicas_Privadas)

<font size="6" color="red">Amostra piloto</font>

# Define margem de erro e grau de confiança

In [ ]:
# Definições gerais
Margem_Erro_Dias = 3
Margem_Erro_ProcurouOutraClinica = 0.05
Margem_Erro_Satisfacao = 0.05
    
Grau_Confianca = 0.95

# Se tiver informações sobre a população, estima variância e proporções

In [ ]:
# Definições específicas (quando já se conhece a população)
Variancia_Dias_Estimada = 506
Proporcao_Procurou_Outra_Clinica_Amostra = 0.3


# Se não tiver informações sobre a população, estima variância e proporções em amostra piloto (mínimo 30 elementos)

In [ ]:
# Definições específicas (quando faz amostra piloto)
Tamanho_Amostra = 30
amostra = recupera_amostra(populacao, Tamanho_Amostra)
Variancia_Dias_Estimada = amostra.Dias_Ate_Atendimento.var()
Proporcao_Procurou_Outra_Clinica_Amostra = sum(amostra.Procurou_Outra_Clinica == 'S')/Tamanho_Amostra
print('Amostra {}, variância dias {}, proporção procurou outra clínica {}'.format(
                                                            Tamanho_Amostra,
                                                            Variancia_Dias_Estimada,
                                                            Proporcao_Procurou_Outra_Clinica_Amostra))

# Cálcula tamanho amostra para Dias

In [ ]:
z = calcula_Z_normal(Grau_Confianca)

Tamanho_Amostra_Dias = Tamanho_Amostra_Valor_Pontual(
    Tamanho_Populacao,
    z,
    Variancia_Dias_Estimada,
    Margem_Erro_Dias)

Tamanho_Amostra_Dias

# Calcula tamanho amostra para procurou outra clínica

In [ ]:
z = calcula_Z_normal(Grau_Confianca)

Tamanho_Amostra_Procurou_Outra_Clinica = Tamanho_Amostra_Proporcao_Simples(
    Tamanho_Populacao,
    z,
    Proporcao_Procurou_Outra_Clinica_Amostra,
    Margem_Erro_ProcurouOutraClinica
)

Tamanho_Amostra_Procurou_Outra_Clinica

# Calcula tamanho da amostra para satisfação

In [ ]:
Tamanho_Amostra_Satisfacao = Tamanho_Amostra_Proporcao_Multipla(
    Tamanho_Populacao,
    Grau_Confianca,
    3,
    Margem_Erro_Satisfacao
)

Tamanho_Amostra_Satisfacao

# Resumo

In [ ]:
print('Tamanho_Amostra_Dias {}, Tamanho_Amostra_Procurou_Outra_Clinica {}, Tamanho_Amostra_Satisfacao {}'.format(
    Tamanho_Amostra_Dias, 
    Tamanho_Amostra_Procurou_Outra_Clinica, 
    Tamanho_Amostra_Satisfacao))

<font size="6" color="red">Inferência dos parâmetros populacionais com base numa amostra</font>

## Cálculo intervalo de confiança para dias

In [ ]:
amostraDias = recupera_amostra(populacao, Tamanho_Amostra_Dias)
amostraDias = amostraDias.Dias_Ate_Atendimento

In [ ]:
amostraDias.var()

In [ ]:
confianca = 0.95

inferior,superior = calcula_intervalo_media(amostraDias, confianca, Tamanho_Populacao)
print('Intervalo da média {} {}'.format(inferior,superior))

In [ ]:
print('Margem de erro {}'.format((superior-inferior)/2))

## Cálculo intervalo de confiança para procurou outra clínica

In [ ]:
amostraProcurouOutraClinica = recupera_amostra(populacao, Tamanho_Amostra_Procurou_Outra_Clinica)
amostraProcurouOutraClinica = amostraProcurouOutraClinica.Procurou_Outra_Clinica

In [ ]:
confianca = 0.95

inferior,superior = calcula_intervalo_proporcao(amostraProcurouOutraClinica, confianca, 'S', Tamanho_Populacao)
print('Intervalo da média {} {}'.format(inferior,superior))

In [ ]:
print('Margem de erro {}'.format((superior-inferior)/2))

## Cálculo intervalo de confiança para satisfação

In [ ]:
amostraSatisfacao = recupera_amostra(populacao, Tamanho_Amostra_Satisfacao)
amostraSatisfacao = amostraSatisfacao.Satisfacao

In [ ]:
confianca = 0.95

inferior,superior = calcula_intervalo_proporcao(amostraSatisfacao, confianca, 'I', Tamanho_Populacao)
print('Intervalo da média {} {}'.format(inferior,superior))

In [ ]:
print('Margem de erro {}'.format((superior-inferior)/2))

<font size="6" color="red">Fazendo amostra estratificada</font>

In [ ]:
proporcao_publico = Tamanho_Populacao_Clinicas_Publicas/Tamanho_Populacao

proporcao_privado = Tamanho_Populacao_Clinicas_Privadas/Tamanho_Populacao

In [ ]:
amostraDiasPublico = recupera_amostra(Populacao_Clinicas_Publicas, proporcao_publico*Tamanho_Amostra_Dias)
amostraDiasPrivado = recupera_amostra(Populacao_Clinicas_Privadas, proporcao_privado*Tamanho_Amostra_Dias)

In [ ]:
amostraDias = pd.concat([amostraDiasPublico,amostraDiasPrivado]).Dias_Ate_Atendimento
len(amostraDias)

In [ ]:
confianca = 0.95

inferior,superior = calcula_intervalo_media(amostraDias, confianca, Tamanho_Populacao)
print('Intervalo da média {} {}'.format(inferior,superior))

In [ ]:
print('Margem de erro {}'.format((superior-inferior)/2))

<font size="6" color="red">Exercicio 4 - AvaliacaoPoliticasPublicas</font>

https://www.bbc.com/portuguese/brasil-45780864

# Criação da população de eleitores simulada

Eleitores divididos em cinco regiões.

Três candidatos concorrendo.

In [ ]:
eleitoresRegiao = [21389000,63887000,11555000,39230000,10595000]
populacao = pd.DataFrame( columns=['Regiao','Voto'])


populacao.Voto = \
['B']*int(0.57*eleitoresRegiao[0]) + ['H']*int(0.20*eleitoresRegiao[0]) + ['C']*int(0.09*eleitoresRegiao[0]) +\
['B']*int(0.53*eleitoresRegiao[1]) + ['H']*int(0.19*eleitoresRegiao[1]) + ['C']*int(0.12*eleitoresRegiao[1]) +\
['B']*int(0.43*eleitoresRegiao[2]) + ['H']*int(0.37*eleitoresRegiao[2]) + ['C']*int(0.09*eleitoresRegiao[2]) +\
['B']*int(0.26*eleitoresRegiao[3]) + ['H']*int(0.51*eleitoresRegiao[3]) + ['C']*int(0.17*eleitoresRegiao[3]) +\
['B']*int(0.58*eleitoresRegiao[4]) + ['H']*int(0.21*eleitoresRegiao[4]) + ['C']*int(0.10*eleitoresRegiao[4]) 

populacao.Regiao = ['Sul']*(int(0.57*eleitoresRegiao[0]) + int(0.20*eleitoresRegiao[0]) + int(0.09*eleitoresRegiao[0])) + \
['Sudeste']*(int(0.53*eleitoresRegiao[1]) + int(0.19*eleitoresRegiao[1]) + int(0.12*eleitoresRegiao[1])) + \
['Norte']*(int(0.43*eleitoresRegiao[2]) + int(0.37*eleitoresRegiao[2]) + int(0.09*eleitoresRegiao[2])) + \
['Nordeste']*(int(0.26*eleitoresRegiao[3]) + int(0.51*eleitoresRegiao[3]) + int(0.17*eleitoresRegiao[3])) + \
['CentroOeste']*(int(0.58*eleitoresRegiao[4]) + int(0.21*eleitoresRegiao[4]) + int(0.10*eleitoresRegiao[4])) 

In [ ]:
TamanhoPopulacao = len(populacao)

In [ ]:
populacao = shuffle(populacao)
populacao.head()

In [ ]:
populacao['Regiao'].value_counts().plot(kind='bar')
plt.xlabel('Regiao')
plt.ylabel('Quantidade')

plt.title('Regiao')
plt.grid(True)
plt.show()



In [ ]:
populacao['Regiao'].value_counts()/TamanhoPopulacao

In [ ]:
populacao['Voto'].value_counts().plot(kind='bar')
plt.xlabel('Voto')
plt.ylabel('Quantidade')

plt.title('Voto')
plt.grid(True)
plt.show()

In [ ]:
populacao['Voto'].value_counts()/TamanhoPopulacao

## Defina o tamanho da amostra

In [ ]:
Grau_Confianca = 0.95
Margem_Erro = 0.02


Tamanho_Amostra_Eleicao = Tamanho_Amostra_Proporcao_Multipla( TamanhoPopulacao,    0.95,    3,    0.02)
print ("Tamanho da Amostra da eleição {}:". format (Tamanho_Amostra_Eleicao))



## Recupere uma amostra com o tamanho acima

In [ ]:
amostra = recupera_amostra (populacao.Voto, Tamanho_Amostra_Eleicao)
print ("Amostra {}:". format (amostra))


## Calcule o intervalo de confiânça para o candidato 'B'

In [ ]:
inferior,superior = calcula_intervalo_proporcao(amostra, 0.95, 'B', TamanhoPopulacao)
print('Intervalo de confiança B {} {}'.format(inferior,superior))

print('Margem de erro {}'.format((superior-inferior)/2))

## Calcule o intervalo de confiânça para o candidato 'H'

In [ ]:
inferior,superior = calcula_intervalo_proporcao(amostra, 0.95, 'H', TamanhoPopulacao)
print('Intervalo de confiança H {} {}'.format(inferior,superior))

print('Margem de erro {}'.format((superior-inferior)/2))

## Calcule o intervalo de confiânça para o candidato 'C'

In [ ]:
inferior,superior = calcula_intervalo_proporcao(amostra, 0.95, 'C', TamanhoPopulacao)
print('Intervalo de confiança C {} {}'.format(inferior,superior))

print('Margem de erro {}'.format((superior-inferior)/2))